<a href="https://colab.research.google.com/github/rohitnikam1/Econometrics/blob/master/13_The_Dangers_of_Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **What is overfitting?**
When constructing a model, we tune both the parameters and the model by fitting to sample data. We then use the model and parameters to predict data we have not yet observed. We say a model is overfit when it is overly sensitive to noise and idiosyncracies in the sample data, and therefore does not reflect the underlying data-generating process.

To understand why this happens, one has to consider the amount of noise present in any dataset. One can consider a set of data as
, the true underlying data that came from whatever process we are trying to model, and , some random noise. Because what we see is $D = D_T + ϵ$, we might fit our model to very perfectly predict for the given $ϵ$, but not for $D$.



This is problematic because we only care about fitting to the sample insofar as that gives an accurate fit to future data. The two broad causes of overfitting are:

- small sample size, so that noise and trend are not distinguishable
- choosing an overly complex model, so that it ends up contorting to fit the noise in the sample

